<a href="https://colab.research.google.com/github/gkarthik29/Analyze-invoice-or-payslip-using-gemini/blob/main/payslip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai

In [3]:
import google.generativeai as genai

In [4]:
#api key
from google.colab import userdata
genai.configure(
    api_key=userdata.get('GAPI_KEY')
)

In [5]:
#see models in key
for i in genai.list_models():
  if 'generateContent' in i.supported_generation_methods:
    print(i.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


In [6]:
#model config to make model to be specific
MODEL_CONFIG = {
    'temperature': 0.1,
    #'candidate_count': 1,
    'top_k': 40,
    'top_p': 0.9,
    'max_output_tokens':4096,
    }

#default safety settings by google
safety_settings = [
    {
        "category": "harm_category_sexually_explicit",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "harm_category_sexually_explicit",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "harm_category_sexually_explicit",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "harm_category_dangerous_content",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    }
]


In [7]:
#what type of model required for our usecase
#for our usecase we need vision model because we need to scan a document and retrieve info from that document

In [8]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-exp-0801",
                              generation_config = MODEL_CONFIG,
                              safety_settings = safety_settings)

In [9]:
#this code is to get pictures and convert store them as byte format
from pathlib import Path

def image_format(image_path):
  img = Path(image_path)

  if not img.exists():
    raise FileNotFoundError(f"File {img} does not exist")

  image_parts = [
      {
          "mime_type":"image/png",  "data" :img.read_bytes()
      }
  ]
  return image_parts

MODEL OUTPUT

In [10]:
def gemini_output(image_path , system_prompt , user_prompt):
  image_parts = image_format(image_path)
  input_prompt = [system_prompt, user_prompt,image_parts[0]]
  response = model.generate_content(input_prompt)
  return response.text

In [11]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               explain it in a neater way.
               """
image_path = "/content/sample payslip.jpg"
user_prompt = "give info in key value pairs in dictionary"
gemini_output(image_path,system_prompt,user_prompt)

'Okay, here\'s the information from the receipt extracted into a dictionary with key-value pairs:\n\n```python\nreceipt_data = {\n    "Employee Name": "A Sample",\n    "Employee Address": "27 The Street\\nLondon\\nSE8 7AB",\n    "Department": "767",\n    "NI No / NI Letter": "AB123456B / A",\n    "Tax Code": "1257L",\n    "PAYE Reference": "034/B859",\n    "Date": "28/07/2021",\n    "Month No": "04",\n    "Pay Method": "BACS",\n    "Contracted Hours": "37.50",\n    "Gross Pay YTD": "6,542.14",\n    "Taxable Pay YTD": "10,676.26",\n    "Tax Paid YTD": "1,296.60",\n    "EE NIC YTD": "957.51",\n    "EE Pension YTD": "490.98",\n    "ER Pension YTD": "994.39",\n    "Period Gross Pay": "3,474.98",\n    "Period Taxable Pay": "3,344.39",\n    "Period NI\'able Pay": "3,474.98",\n    "Period Net Pay": "2,563.83",\n    "Period ER NIC": "377.84",\n    "Period ER Pension": "264.49",\n    "Salary": "1,691.92",\n    "Sleep in Supplement (3.00 units)": "44.85",\n    "Sleep in Supplement (10.00 units)"

In [14]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """
#system_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "
image_path = "/content/sample payslip.jpg"
user_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "

In [15]:
gemini_output(image_path,system_prompt,user_prompt)

'```json\n{\n "employer_name": "A Sample",\n "employer_address": "27 The Street\\nLondon\\nSE8 7AB",\n "employee_address": null,\n "department": "767",\n "ni_no": "AB123456B",\n "ni_letter": "A",\n "tax_code": "1257L",\n "paye_reference": "034/B859",\n "date": "28/07/2021",\n "month_no": "04",\n "pay_method": "BACS",\n "contracted_hours": "37.50",\n "gross_pay_ytd": "6,542.14",\n "taxable_pay_ytd": "10,676.26",\n "tax_paid_ytd": "1,296.60",\n "ee_nic_ytd": "957.51",\n "ee_pension_ytd": "490.98",\n "er_pension_ytd": "994.39",\n "period_gross_pay": "3,474.98",\n "period_taxable_pay": "3,344.39",\n "period_ni\'able_pay": "3,474.98",\n "period_net_pay": "2,563.83",\n "period_er_nic": "377.84",\n "period_er_pension": "264.49",\n "payments": [\n  {\n   "description": "Salary",\n   "unit": null,\n   "rate": null,\n   "amount": "1,691.92"\n  },\n  {\n   "description": "Sleep in Supplement",\n   "unit": "3.00",\n   "rate": "14.95",\n   "amount": "44.85"\n  },\n  {\n   "description": "Sleep in S

In [16]:
from IPython.display import Markdown
Markdown(gemini_output(image_path,system_prompt,user_prompt))


```json
{
 "employer_name": "A Sample",
 "employer_address": "27 The Street\nLondon\nSE8 7AB",
 "employee_address": null,
 "department": "767",
 "ni_no": "AB123456B",
 "ni_letter": "A",
 "tax_code": "1257L",
 "paye_reference": "034/B859",
 "date": "28/07/2021",
 "month_no": "04",
 "pay_method": "BACS",
 "contracted_hours": "37.50",
 "gross_pay_ytd": "6,542.14",
 "taxable_pay_ytd": "10,676.26",
 "tax_paid_ytd": "1,296.60",
 "ee_nic_ytd": "957.51",
 "ee_pension_ytd": "490.98",
 "er_pension_ytd": "994.39",
 "period_gross_pay": "3,474.98",
 "period_taxable_pay": "3,344.39",
 "period_ni'able_pay": "3,474.98",
 "period_net_pay": "2,563.83",
 "period_er_nic": "377.84",
 "period_er_pension": "264.49",
 "payments": [
  {
   "description": "Salary",
   "unit": null,
   "rate": null,
   "amount": "1,691.92"
  },
  {
   "description": "Sleep in Supplement",
   "unit": "3.00",
   "rate": "14.95",
   "amount": "44.85"
  },
  {
   "description": "Sleep in Supplement",
   "unit": "10.00",
   "rate": "14.95",
   "amount": "149.50"
  },
  {
   "description": "Sleep in Supplement",
   "unit": "1.00",
   "rate": "14.95",
   "amount": "14.95"
  },
  {
   "description": "Additional Hours",
   "unit": "45.50",
   "rate": "10.41",
   "amount": "473.66"
  },
  {
   "description": "Additional Hours",
   "unit": "23.50",
   "rate": "10.41",
   "amount": "244.64"
  },
  {
   "description": "Additional Hours",
   "unit": "7.50",
   "rate": "10.41",
   "amount": "78.08"
  },
  {
   "description": "Holiday Pay",
   "unit": "37.50",
   "rate": "3.93",
   "amount": "147.38"
  },
  {
   "description": "Sleep In",
   "unit": "14.00",
   "rate": "45.00",
   "amount": "630.00"
  },
  {
   "description": "Employee Pension",
   "unit": null,
   "rate": null,
   "amount": "-130.59"
  }
 ],
 "deductions": [
  {
   "description": "Income tax",
   "amount": "-459.20"
  },
  {
   "description": "NI Contribution",
   "amount": "-321.36"
  }
 ],
 "taxable_pay": "3,344.39",
 "net_payment": "2,563.83"
}
```